In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import altair as alt

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
SELECT o.orderid, o.extorderid, ol.sku, ol.quantity, o.extordercreatedat FROM PIPE17.PIPE17_RAW.ORDERS o JOIN PIPE17.PIPE17_RAW.ORDERS_LINES ol ON o.ORDERID = ol.ORDERID

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
df = cell2.to_pandas()

# Convert extordercreatedat to datetime
df['EXTORDERCREATEDAT'] = pd.to_datetime(df['EXTORDERCREATEDAT'])

# Extract date from datetime
df['date'] = df['EXTORDERCREATEDAT'].dt.date

# Group by date and SKU to get total quantity per day per SKU
daily_sales = df.groupby(['date', 'SKU'])['QUANTITY'].sum().reset_index()

# Streamlit app
st.title('Top N SKUs Sales per Day')

# How many SKUs to check for top
N = 10

# Identify top N SKUs
top_skus = daily_sales.groupby('SKU')['QUANTITY'].sum().nlargest(N).index

# Separate top N SKUs and 'Others'
daily_sales['SKU'] = daily_sales['SKU'].where(daily_sales['SKU'].isin(top_skus), 'Others')

# Aggregate the 'Others'
daily_sales_agg = daily_sales.groupby(['date', 'SKU'])['QUANTITY'].sum().reset_index()

# Plotting
chart = alt.Chart(daily_sales_agg).mark_bar().encode(
    x='date:T',
    y='QUANTITY:Q',
    color='SKU:N',
    tooltip=['date:T', 'SKU:N', 'QUANTITY:Q']
).properties(
    width=800,
    height=400
)

st.altair_chart(chart, use_container_width=True)

# Initial inventory level
initial_inventory = 1000

# Step 1: Calculate average daily sales for each SKU
average_daily_sales = daily_sales_agg.groupby('SKU')['QUANTITY'].mean().reset_index()
average_daily_sales.columns = ['SKU', 'average_daily_sales']

# Step 2: Estimate days to deplete inventory
average_daily_sales['days_to_deplete'] = initial_inventory / average_daily_sales['average_daily_sales']

# Filter for top SKUs
inventory_forecast = average_daily_sales[average_daily_sales['SKU'].isin(top_skus)].reset_index(drop=True)

# Display the forecast table
st.subheader('Inventory Depletion Forecast for Top SKUs')
st.write(inventory_forecast)